# Feature Engineering

In this notebook we create **9 feature sets** from the processed dataset:

1. **Recursive Feature Elimination**
2. **Select K Best**
3. **Fisher Score Chi-Square**
4. **Extra Trees Classifier**
5. **Pearson Correlation**
6. **Mutual Information**
7. **Mutual Info Regression**
8. **Manual Uniqueness**
9. **Variance Threshold**

For each method select:
- 7 features from PSS-10
- 5 features from GAD-7
- 6 features from PHQ-9

Combine to 18 features → standardize

We then split the data (80/20, stratified) for model training later.

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import (
    RFE, SelectKBest, chi2, VarianceThreshold, f_classif,
    mutual_info_classif, mutual_info_regression
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from scipy.stats import pearsonr

DATA_IN = Path("../data/processed/mhp_processed.csv")
OUT_BASE = Path("../data/processed/features1")
OUT_BASE.mkdir(parents=True, exist_ok=True)

SUMMARY_PATH = Path("../data/processed/feature_selection_summary_v2.csv")

RANDOM_STATE = 42

## Load and prepare the dataset

In [2]:
df = pd.read_csv(DATA_IN)
print("Loaded:", DATA_IN, "shape:", df.shape)

pss_cols = [c for c in df.columns if c.upper().startswith("PSS")]
gad_cols = [c for c in df.columns if c.upper().startswith("GAD")]
phq_cols = [c for c in df.columns if c.upper().startswith("PHQ")]

print("Detected columns counts:", len(pss_cols), len(gad_cols), len(phq_cols))
print("PSS cols:", pss_cols)
print("GAD cols:", gad_cols)
print("PHQ cols:", phq_cols)

if "DepressionEncoded" not in df.columns or df["DepressionEncoded"].isna().sum() > 0:
    df_phq = df[phq_cols].apply(pd.to_numeric, errors="coerce").fillna(0)
    phq_sum = df_phq.sum(axis=1)
    df["DepressionEncoded"] = pd.cut(
        phq_sum,
        bins=[-1, 4, 9, 14, 19, 27],
        labels=[0,1,2,3,4]
    ).astype(int)
print("Target distribution (DepressionEncoded):")
print(df["DepressionEncoded"].value_counts())

Loaded: ..\data\processed\mhp_processed.csv shape: (2022, 34)
Detected columns counts: 10 7 9
PSS cols: ['PSS1', 'PSS2', 'PSS3', 'PSS4', 'PSS5', 'PSS6', 'PSS7', 'PSS8', 'PSS9', 'PSS10']
GAD cols: ['GAD1', 'GAD2', 'GAD3', 'GAD4', 'GAD5', 'GAD6', 'GAD7']
PHQ cols: ['PHQ1', 'PHQ2', 'PHQ3', 'PHQ4', 'PHQ5', 'PHQ6', 'PHQ7', 'PHQ8', 'PHQ9']
Target distribution (DepressionEncoded):
DepressionEncoded
3    511
4    504
2    455
1    411
0    141
Name: count, dtype: int64


## Feature Selection Helpers

In [3]:
def select_rfe(X, y, k):
    model = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)
    sel = RFE(model, n_features_to_select=k)
    sel.fit(X, y)
    return X.columns[sel.support_].tolist()

def select_skb(X, y, k):
    sel = SelectKBest(score_func=f_classif, k=k)
    sel.fit(X.fillna(0), y)
    return X.columns[sel.get_support()].tolist()

def select_fscs(X, y, k):
    Xn = X.copy()
    Xn = Xn - Xn.min()
    sel = SelectKBest(score_func=chi2, k=k)
    sel.fit(Xn.fillna(0).astype(int), y)
    return X.columns[sel.get_support()].tolist()

def select_etc(X, y, k):
    model = ExtraTreesClassifier(n_estimators=200, random_state=RANDOM_STATE)
    model.fit(X.fillna(0), y)
    importances = pd.Series(model.feature_importances_, index=X.columns)
    return importances.nlargest(k).index.tolist()

def select_pc(X, y, k):
    scores = []
    for c in X.columns:
        try:
            r = pearsonr(X[c].fillna(X[c].mean()), y)[0]
        except Exception:
            r = 0.0
        scores.append((abs(r), c))
    scores.sort(reverse=True)
    return [c for _, c in scores[:k]]

def select_mi(X, y, k):
    scores = mutual_info_classif(X.fillna(0), y, random_state=RANDOM_STATE)
    order = np.argsort(scores)[::-1][:k]
    return X.columns[order].tolist()

def select_mir(X, y, k):
    scores = mutual_info_regression(X.fillna(0), y, random_state=RANDOM_STATE)
    order = np.argsort(scores)[::-1][:k]
    return X.columns[order].tolist()

def select_mu(X, y, k):
    uniq = X.nunique().sort_values(ascending=False)
    return uniq.head(k).index.tolist()

def select_vt(X, y, k):
    var = X.var().sort_values(ascending=False)
    return var.head(k).index.tolist()

## Run all nine feature selection methods

In [4]:
methods = {
    "rfe": select_rfe,
    "skb": select_skb,
    "fscs": select_fscs,
    "etc": select_etc,
    "pc": select_pc,
    "mi": select_mi,
    "mir": select_mir,
    "mu": select_mu,
    "vt": select_vt
}

summary = []
for name, func in methods.items():
    print(f"\n➡ Running method: {name.upper()}")

    selected = []
    sel_pss = func(df[pss_cols], df["DepressionEncoded"], k=7)
    sel_gad = func(df[gad_cols], df["DepressionEncoded"], k=5)
    sel_phq = func(df[phq_cols], df["DepressionEncoded"], k=6)

    selected.extend(sel_pss)
    selected.extend(sel_gad)
    selected.extend(sel_phq)
    print(f"Selected ({len(selected)}):", selected)

    X_sel = df[selected].copy()
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X_sel.fillna(0)), columns=selected)

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, df["DepressionEncoded"], test_size=0.2, random_state=RANDOM_STATE, stratify=df["DepressionEncoded"]
    )

    out_dir = OUT_BASE / name
    out_dir.mkdir(parents=True, exist_ok=True)
    train_df = X_train.copy()
    train_df["DepressionEncoded"] = y_train.values
    test_df = X_test.copy()
    test_df["DepressionEncoded"] = y_test.values

    train_df.to_csv(out_dir / "train.csv", index=False)
    test_df.to_csv(out_dir / "test.csv", index=False)

    summary.append({"Method": name, "Selected Features": selected})


➡ Running method: RFE
Selected (18): ['PSS1', 'PSS2', 'PSS3', 'PSS4', 'PSS5', 'PSS9', 'PSS10', 'GAD1', 'GAD3', 'GAD4', 'GAD6', 'GAD7', 'PHQ2', 'PHQ4', 'PHQ5', 'PHQ6', 'PHQ7', 'PHQ8']

➡ Running method: SKB
Selected (18): ['PSS1', 'PSS2', 'PSS3', 'PSS4', 'PSS6', 'PSS9', 'PSS10', 'GAD1', 'GAD4', 'GAD5', 'GAD6', 'GAD7', 'PHQ2', 'PHQ3', 'PHQ4', 'PHQ6', 'PHQ7', 'PHQ8']

➡ Running method: FSCS
Selected (18): ['PSS1', 'PSS2', 'PSS3', 'PSS4', 'PSS6', 'PSS9', 'PSS10', 'GAD1', 'GAD4', 'GAD5', 'GAD6', 'GAD7', 'PHQ2', 'PHQ3', 'PHQ6', 'PHQ7', 'PHQ8', 'PHQ9']

➡ Running method: ETC
Selected (18): ['PSS8', 'PSS5', 'PSS6', 'PSS7', 'PSS9', 'PSS1', 'PSS4', 'GAD7', 'GAD3', 'GAD4', 'GAD2', 'GAD1', 'PHQ6', 'PHQ2', 'PHQ9', 'PHQ4', 'PHQ3', 'PHQ7']

➡ Running method: PC
Selected (18): ['PSS10', 'PSS2', 'PSS3', 'PSS4', 'PSS1', 'PSS9', 'PSS8', 'GAD7', 'GAD4', 'GAD5', 'GAD1', 'GAD6', 'PHQ2', 'PHQ6', 'PHQ4', 'PHQ7', 'PHQ3', 'PHQ8']

➡ Running method: MI
Selected (18): ['PSS10', 'PSS1', 'PSS4', 'PSS9', 'PSS3', 'P

## Save the Summary

In [5]:
pd.DataFrame(summary).to_csv(SUMMARY_PATH, index=False)
print("\n✅ Feature selection complete. Summary saved to:", SUMMARY_PATH)


✅ Feature selection complete. Summary saved to: ..\data\processed\feature_selection_summary_v2.csv


## Summary of Selected Features

In [6]:
summary_path = Path("../data/processed/feature_selection_summary_v2.csv")

if summary_path.exists():
    summary_df = pd.read_csv(summary_path)
    pd.set_option("display.max_colwidth", None)
    display(summary_df)
else:
    print("⚠️ No summary file found. Please run all feature selection steps first.")

,Method,Selected Features
0,rfe,"['PSS1', 'PSS2', 'PSS3', 'PSS4', 'PSS5', 'PSS9', 'PSS10', 'GAD1', 'GAD3', 'GAD4', 'GAD6', 'GAD7', 'PHQ2', 'PHQ4', 'PHQ5', 'PHQ6', 'PHQ7', 'PHQ8']"
1,skb,"['PSS1', 'PSS2', 'PSS3', 'PSS4', 'PSS6', 'PSS9', 'PSS10', 'GAD1', 'GAD4', 'GAD5', 'GAD6', 'GAD7', 'PHQ2', 'PHQ3', 'PHQ4', 'PHQ6', 'PHQ7', 'PHQ8']"
2,fscs,"['PSS1', 'PSS2', 'PSS3', 'PSS4', 'PSS6', 'PSS9', 'PSS10', 'GAD1', 'GAD4', 'GAD5', 'GAD6', 'GAD7', 'PHQ2', 'PHQ3', 'PHQ6', 'PHQ7', 'PHQ8', 'PHQ9']"
3,etc,"['PSS8', 'PSS5', 'PSS6', 'PSS7', 'PSS9', 'PSS1', 'PSS4', 'GAD7', 'GAD3', 'GAD4', 'GAD2', 'GAD1', 'PHQ6', 'PHQ2', 'PHQ9', 'PHQ4', 'PHQ3', 'PHQ7']"
4,pc,"['PSS10', 'PSS2', 'PSS3', 'PSS4', 'PSS1', 'PSS9', 'PSS8', 'GAD7', 'GAD4', 'GAD5', 'GAD1', 'GAD6', 'PHQ2', 'PHQ6', 'PHQ4', 'PHQ7', 'PHQ3', 'PHQ8']"
5,mi,"['PSS10', 'PSS1', 'PSS4', 'PSS9', 'PSS3', 'PSS2', 'PSS6', 'GAD7', 'GAD4', 'GAD5', 'GAD6', 'GAD1', 'PHQ2', 'PHQ6', 'PHQ4', 'PHQ7', 'PHQ3', 'PHQ8']"
6,mir,"['PSS4', 'PSS10', 'PSS2', 'PSS1', 'PSS9', 'PSS3', 'PSS5', 'GAD7', 'GAD4', 'GAD5', 'GAD6', 'GAD1', 'PHQ2', 'PHQ6', 'PHQ4', 'PHQ7', 'PHQ9', 'PHQ5']"
7,mu,"['PSS1', 'PSS2', 'PSS3', 'PSS4', 'PSS5', 'PSS6', 'PSS7', 'GAD1', 'GAD2', 'GAD3', 'GAD4', 'GAD5', 'PHQ1', 'PHQ2', 'PHQ3', 'PHQ4', 'PHQ5', 'PHQ6']"
8,vt,"['PSS4', 'PSS1', 'PSS9', 'PSS10', 'PSS2', 'PSS3', 'PSS8', 'GAD7', 'GAD2', 'GAD3', 'GAD6', 'GAD5', 'PHQ9', 'PHQ6', 'PHQ3', 'PHQ5', 'PHQ2', 'PHQ8']"
